In [63]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import re
import math
plt.gray()
fnames = !ls /Users/nakatani/labo/grabcut/img/foreground2_fixed
print(len(fnames))
!pwd

252
/Users/nakatani/labo/grabcut/samples/python


In [115]:
def get_radius(mask_img):
    contours,hierarchy = cv2.findContours(mask_img, 1, 2)
    for i in range(len(contours)):
        if len(contours[i]) > 0:

            # remove small objects
            if cv2.contourArea(contours[i]) < 1500:
                continue
            else:
                cnt = contours[i]
                (x,y),radius = cv2.minEnclosingCircle(cnt)
                return int(radius/5)

In [116]:
def Harris(mask_img, thresh_value):
    
#     thresh_value = 0.02
    
#     gray = cv2.cvtColor(mask_img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(mask_img)
    dst = cv2.cornerHarris(gray,11,11,0.1)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst, dst.max() * thresh_value, 255, cv2.THRESH_BINARY)
    dst = np.uint8(dst)
    _, __, ___, centroids = cv2.connectedComponentsWithStats(dst)
    res = np.int64(centroids)
#     print(res) 
#     mask_img[res[1:,1],res[1:,0]]=[0,0,255]
    coord = res[1:, :].T
    coord = coord[::-1]
#     print(coord) 
    
    return(coord)

In [117]:
def find_IS_coordinate(center, radius, boundary_copy_img):
    '''
    is >> intersection
    
    input:
        center >> - of circle
        radius >> - of circles
        boundary_copy_img >> -
    output:
        cx, cy >> coordinate of a circle center
        cx1, cy1 >> coordinate of intersection1
        cx2, cy2 >> coordinate of intersection2
    '''
    shape = boundary_copy_img.shape
    blank = np.zeros(shape, np.uint8)
    blank = cv2.circle(blank, center, radius, [255,255,0],1)
    boundary_copy_img = cv2.circle(boundary_copy_img, (coord[1][j],coord[0][j]), radius, [255,255,0],1)


    # find IS
    intersection_img = cv2.bitwise_and(boundary_img, blank)
    gray_is = cv2.cvtColor(intersection_img, cv2.COLOR_BGR2GRAY)
    _, gray_is = cv2.threshold(gray_is, 1, 255, cv2.THRESH_BINARY)
    gray_is = cv2.dilate(gray_is,None)
    contours, hierarchy = cv2.findContours(gray_is, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    coord_list = []
    print(len(contours))
    # intersections per acircle == 2
    if (len(contours)) < 2:
        return 0
    for k in range(2):
        M = cv2.moments(contours[k])
        if M['m00'] == 0:
            continue
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        # contour重心(交点)をlistに
        coord_list.append((cx, cy))

    cx, cy = center
    cx1, cy1 = coord_list[0] 
    cx2, cy2 = coord_list[1]
    return (cx, cy, cx1, cy1, cx2, cy2)

In [118]:
def check_convex(img, c1, c2):
    x1 = int(c1[0]*0.8 + c2[0]*0.2)
    y1 = int(c1[1]*0.8 + c2[1]*0.2)    
    x2 = int(c1[0]*0.2 + c2[0]*0.8)    
    y2 = int(c1[1]*0.2 + c2[1]*0.8)
    x3 = int(c1[0]*0.5 + c2[0]*0.5)    
    y3 = int(c1[1]*0.5 + c2[1]*0.5)
    
    if img[y1][x1][0] + img[y2][x2][0] + img[y3][x3][0]== 0:
        return False
    else: return True

In [119]:
def calc_radian(C):
    cx, cy, cx1, cy1, cx2, cy2 = C
    v1 = np.array([cx1 - cx, cy1 - cy])
    v2 = np.array([cx2 - cx, cy2 - cy])

    cos_theta = np.inner(v1, v2) /  (np.linalg.norm(v1) * np.linalg.norm(v2))
    theta = math.acos(np.clip(cos_theta, -1.0, 1.0))
#     print(math.degrees(theta))
#     print(c)
    
    return theta

In [120]:
def synthesis(img1, img2):
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    _,gray_img1 = cv2.threshold(gray_img1,1,255,0)
    _,gray_img2 = cv2.threshold(gray_img2,1,255,0)
    not1 = cv2.bitwise_not(gray_img1)
    not2 = cv2.bitwise_not(gray_img2)
    
    bitwise_and1 = cv2.bitwise_and(img2, img2, mask = not1)
    bitwise_and2 = cv2.bitwise_and(img1, img1, mask = not2)

    synthetic_img_1on2 = cv2.bitwise_or(img1, bitwise_and1)
    synthetic_img_2on1 = cv2.bitwise_or(img2, bitwise_and2)
    
    return synthetic_img_1on2, synthetic_img_2on1

In [121]:
# red: foreground, green: background

def synthesis_mask(img1, img2):
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    _,gray_img1 = cv2.threshold(gray_img1,1,255,0)
    _,gray_img2 = cv2.threshold(gray_img2,1,255,0)
    not1 = cv2.bitwise_not(gray_img1)
    not2 = cv2.bitwise_not(gray_img2)
    gray1in_rbg = cv2.cvtColor(gray_img1, cv2.COLOR_GRAY2BGR)
    gray2in_rbg = cv2.cvtColor(gray_img2, cv2.COLOR_GRAY2BGR)
    
    red = np.ones(patch1.shape,np.uint8)*255
    red[:, :, (1, 2)] = 0
    green = np.ones(patch1.shape,np.uint8)*255
    green[:, :, (0, 2)] = 0


    bground2 = cv2.bitwise_and(gray2in_rbg, green, mask = not1)
    bground1 = cv2.bitwise_and(gray1in_rbg, green, mask = not2)

    fground1 = cv2.bitwise_and(red, red, mask = gray_img1)
    fground2 = cv2.bitwise_and(red, red, mask = gray_img2)

    
    mask_img_1on2 = cv2.bitwise_or(fground1, bground2)
    mask_img_2on1 = cv2.bitwise_or(fground2, bground1)
    
    return mask_img_1on2, mask_img_2on1

In [122]:
#　下のやつはconcaveを取り出すコード

In [124]:
mask = '../../img/mask2/'
boundary = '../../img/boundary2/'
foreground = '../../img/foreground2_fixed/'

l = 20
thresh_value = 0.00001


for i in range(len(fnames)):
    fname = fnames[i]
    fname = '217.png'
    print(fname, i)
    no_ = re.sub('.png', '', fname)
    
    # i >> img, m >> mask
#     if not os.path.exists('../../img/patch2/i' + no_): os.makedirs('../../img/patch2/i' + no_)
#     if not os.path.exists('../../img/patch2/m' + no_): os.makedirs('../../img/patch2/m' + no_)
#     if not os.path.exists('../../img/patch2_convex/i' + no_): os.makedirs('../../img/patch2_convex/i' + no_)
#     if not os.path.exists('../../img/patch2_convex/m' + no_): os.makedirs('../../img/patch2_convex/m' + no_)
    
    
# =====load img======    
    img = cv2.imread(foreground + fname)
    img_copy = img + 0
    img_copy2 = img + 0
    blank1 = np.zeros(img.shape, np.uint8)
    blank2 = np.zeros(img.shape, np.uint8)
    mask_img = cv2.imread(mask + fname, cv2.IMREAD_GRAYSCALE)
    boundary_img = cv2.imread(boundary + fname)
    boundary_copy = boundary_img + 0
# =====load img======    


    radius = get_radius(mask_img)
    coord = Harris(mask_img, thresh_value) # find centroid of corners, then record in coord
    print(coord)
    
    cnt = 0 # for patch#
    
    for j in range(len(coord[0])):
        
#         print(j)
        
        center = (coord[1][j],coord[0][j])
        

        C = find_IS_coordinate(center, int(radius*0.5), boundary_copy) # find intersections, then record in C
#         print(C)
        if(C == 0): continue
        cx, cy, cx1, cy1, cx2, cy2 = C
        
        
        patch1 = img[cy1-l:cy1+l, cx1-l:cx1+l]
        patch2 = img[cy2-l:cy2+l, cx2-l:cx2+l]
        try:
            print('test')
            if(patch1.shape != patch2.shape or patch1.shape[0] * patch1[0].shape[1] == 0):
                print(fname, '('+str(cnt)+')')
                cnt += 2
                continue
        except:
                import traceback
                traceback.print_exc()
                print(fname, '('+str(cnt)+')')
                cnt += 2
                continue
                
#         print('test')
            
        
        if not(check_convex(img, [cx1, cy1], [cx2, cy2])): 
#             print('test')
            img_copy = cv2.circle(img_copy, (cx,cy), 1, [0,255,0],5)
#             img_copy2 = cv2.circle(img_copy2, (cx,cy), 1, [0,255,0],9)
            blank1 = cv2.circle(blank1, (cx,cy), 1, [0,255,0],5)
#             img_copy2 = cv2.rectangle(img_copy2, (cx1-l, cy1-l), (cx1+l, cy1+l), (0, 255, 0), thickness=2)
#             img_copy2 = cv2.rectangle(img_copy2, (cx2-l, cy2-l), (cx2+l, cy2+l), (0, 255, 0), thickness=2)
            
            syn1, syn2 = synthesis(patch1, patch2)
            mask1, mask2 = synthesis_mask(patch1, patch2)
#             cv2.imwrite('../../img/patch2/i' + no_+ '/'  + str(cnt) + '.png', syn1)
#             cv2.imwrite('../../img/patch2/m' + no_ + '/'  + str(cnt) + '.png', mask1)
            cnt += 1
#             cv2.imwrite('../../img/patch2/i' + no_ + '/'  + str(cnt) + '.png', syn2)
#             cv2.imwrite('../../img/patch2/m' + no_ + '/'  + str(cnt) + '.png', mask2)
            cnt += 1
            
        else:
#             print('test')
            img_copy = cv2.circle(img_copy, (cx,cy), 1, [0,0,255],5)
            blank2 = cv2.circle(blank2, (cx,cy), 1, [0,0,255],5)
#             img_copy2 = cv2.circle(img_copy2, (cx,cy), 1, [0,0,255],9)
#             img_copy2 = cv2.rectangle(img_copy2, (cx1-l, cy1-l), (cx1+l, cy1+l), (0, 0, 255), thickness=2)
#             img_copy2 = cv2.rectangle(img_copy2, (cx2-l, cy2-l), (cx2+l, cy2+l), (0, 0, 255), thickness=2)
            
            
            syn1, syn2 = synthesis(patch1, patch2)
            mask1, mask2 = synthesis_mask(patch1, patch2)
#             cv2.imwrite('../../img/patch2_convex/i' + no_+ '/'  + str(cnt) + '.png', syn1)
#             cv2.imwrite('../../img/patch2_convex/m' + no_ + '/'  + str(cnt) + '.png', mask1)
            cnt += 1
#             cv2.imwrite('../../img/patch2_convex/i' + no_ + '/'  + str(cnt) + '.png', syn2)
#             cv2.imwrite('../../img/patch2_convex/m' + no_ + '/'  + str(cnt) + '.png', mask2)
            cnt += 1
        
        


        
        # cx, cy, cx1, cy1, cx2, cy2 から角度計算
    
#     plt.figure(figsize = (29,20))
#     plt.imshow(img_copy), plt.show()
#     plt.figure(figsize = (29,20))
#     plt.imshow(img_copy2), plt.show()
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    cv2.imwrite('../../img/corner2/corner/' + fname, img_copy)
    cv2.imwrite('../../img/corner2/concave/' + fname, blank1)
    cv2.imwrite('../../img/corner2/convex/' + fname, blank2)
#     if i > 10: break
    print(radius, "=====")
    break


217.png 0
[[176 176 192 219 235 250 412 446 465 494 513 536 566 576 606 642 663 702
  725 746 788 837 841 854 859 863]
 [553 751 592 471 673 648 426 825 367 391 402 362 332 322 302 957 732 388
  443 433 476 532 554 640 583 616]]
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
2
test
71 =====


/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/synthe/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


In [ ]:
a = [1,2,3,4,5]

In [205]:
b = a[1:3]
b

[2, 3]

In [41]:
b[0]

2

In [61]:
mask = '../../img/mask2/'
boundary = '../../img/boundary2/'
foreground = '../../img/foreground2_fixed/'

l = 20
thresh_value = 0.0001


for i in range(len(fnames)):
    i = 247
    fname = fnames[i]
    print(fname, i)
    no_ = re.sub('.png', '', fname)
    
    # i >> img, m >> mask
    if not os.path.exists('../../img/patch2/i' + no_): os.makedirs('../../img/patch2/i' + no_)
    if not os.path.exists('../../img/patch2/m' + no_): os.makedirs('../../img/patch2/m' + no_)
    if not os.path.exists('../../img/patch2_convex/i' + no_): os.makedirs('../../img/patch2_convex/i' + no_)
    if not os.path.exists('../../img/patch2_convex/m' + no_): os.makedirs('../../img/patch2_convex/m' + no_)
    
    
# =====load img======    
    img = cv2.imread(foreground + fname)
    img_copy = img + 0
    img_copy2 = img + 0
    mask_img = cv2.imread(mask + fname, cv2.IMREAD_GRAYSCALE)
    boundary_img = cv2.imread(boundary + fname)
    boundary_copy = boundary_img + 0
# =====load img======    


    radius = get_radius(mask_img)
    coord = Harris(mask_img, thresh_value) # find centroid of corners, then record in coord
    
    cnt = 0 # for patch#
    
    for j in range(len(coord[0])):
        
        center = (coord[1][j],coord[0][j])
        

        C = find_IS_coordinate(center, int(radius*0.5), boundary_copy) # find intersections, then record in C
        if(C == 0): continue
        cx, cy, cx1, cy1, cx2, cy2 = C
        
        
        patch1 = img[cy1-l:cy1+l, cx1-l:cx1+l]
        patch2 = img[cy2-l:cy2+l, cx2-l:cx2+l]
        if(patch1.shape != patch2.shape):
            print(fname, '('+str(cnt)+')')
            cnt += 2
            continue
        print(patch1.shape)
            
        
        if not(check_convex(img, [cx1, cy1], [cx2, cy2])): 
            img_copy = cv2.circle(img_copy, (cx,cy), 1, [0,255,0],9)
            
            syn1, syn2 = synthesis(patch1, patch2)
            mask1, mask2 = synthesis_mask(patch1, patch2)
            cv2.imwrite('../../img/patch2/i' + no_+ '/'  + str(cnt) + '.png', syn1)
            cv2.imwrite('../../img/patch2/m' + no_ + '/'  + str(cnt) + '.png', mask1)
            cnt += 1
            cv2.imwrite('../../img/patch2/i' + no_ + '/'  + str(cnt) + '.png', syn2)
            cv2.imwrite('../../img/patch2/m' + no_ + '/'  + str(cnt) + '.png', mask2)
            cnt += 1
            
        else:
            img_copy = cv2.circle(img_copy, (cx,cy), 1, [0,0,255],9)

            syn1, syn2 = synthesis(patch1, patch2)
            mask1, mask2 = synthesis_mask(patch1, patch2)
            cv2.imwrite('../../img/patch2/i' + no_+ '/'  + str(cnt) + '.png', syn1)
            cv2.imwrite('../../img/patch2/m' + no_ + '/'  + str(cnt) + '.png', mask1)
            cnt += 1
            cv2.imwrite('../../img/patch2_convex/i' + no_ + '/'  + str(cnt) + '.png', syn2)
            cv2.imwrite('../../img/patch2_convex/m' + no_ + '/'  + str(cnt) + '.png', mask2)
            cnt += 1
        
        


        
        # cx, cy, cx1, cy1, cx2, cy2 から角度計算
    
    plt.figure(figsize = (29,20))
    
    plt.imshow(img_copy), plt.show()
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    cv2.imwrite('./img/' + fname, img_copy)
    cv2.imwrite('../../img/patch2/whole/' + fname, img_copy)
#     if i > 10: break
    break

95.png 247
(0, 40, 3)


/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/synthe/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


error: OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-_xfu06ks/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
